In [1]:
import daria as da
import cv2
import numpy as np
#import greyscale image
# im = cv2.imread("../images/originals/Baseline.jpg", 0)

# img = da.Image(im, [0,0], 280, 150)
# img.show()

# img.coarsen(0.001,0.001)
# img.show()



def backward_diff_x(im):
    img = im.astype(float)
    return np.c_[img,img[:,-1]][:,1:]-img

def forward_diff_x(im):
    img = im.astype(float)
    return img - np.c_[img[:,0],img][:,:-1]

def laplace_x(im):
    return 0.5*(forward_diff_x(backward_diff_x(im))+backward_diff_x(forward_diff_x(im)))

def backward_diff_y(im):
    img = im.astype(float)
    return np.r_[img,[img[-1,:]]][1:,:]-img

def forward_diff_y(im):
    img = im.astype(float)
    return img - np.r_[[img[0,:]],img][:-1,:]

def laplace_y(im):
    return 0.5*(forward_diff_y(backward_diff_y(im))+backward_diff_y(forward_diff_y(im)))

def laplace(im):
    return laplace_x(im)+laplace_y(im)


from math import sqrt

def norm(x):
    return sqrt(np.tensordot(x,x))

def richardson(operator, b, im0, tol, omega, max_iter= 1000):
    res = b-operator(im0)
    im = im0
    nr = norm(res)
    iteration = 0
    while nr>tol and iteration<max_iter:
        im = im + omega*res
        res = b - operator(im)
        nr = norm(res)
        print(nr)
        iteration += 1
    print(iteration)

    return im


def op(im):
    return im - 0.1*laplace(im)


def restriction(im, diff = 0.5):
    return cv2.resize(im, (0,0), fx = diff, fy = diff)

def prolong(im, dim):
    #return cv2.resize(im, (0,0), fx = diff, fy = diff)
    return cv2.resize(im, dim)

def multigrid_richardson(op, b, im0, tol, rich_param, max_iter = 100, depth = 3):
    
    # pre-smoothing 
    im = richardson(op, b, im0, tol=0.1, omega = rich_param, max_iter = 1)
    
    residual = b-op(im)

    # restriction
    rhs = restriction(residual)

    # error-correction
    eps = richardson(op, rhs, rhs, tol = 0.01, omega=rich_param, max_iter = 1000)

    # prolongation
    im += prolong(eps, (residual.shape[1],residual.shape[0]))


    # post-smoothing
    im = richardson(op, b, im, tol=0.1, omega = rich_param, max_iter = 1)


    return im


def multigrid_richardson(op, b, im0, tol, rich_param, max_iter = 100, depth = 3):
    
    # pre-smoothing 
    im = richardson(op, b, im0, tol=0.1, omega = rich_param, max_iter = 1)
    
    residual = b-op(im)

    # restriction
    rhs = restriction(residual)

    # error-correction
    eps = richardson(op, rhs, rhs, tol = 0.01, omega=rich_param, max_iter = 1000)

    # prolongation
    im += prolong(eps, (residual.shape[1],residual.shape[0]))


    # post-smoothing
    im = richardson(op, b, im, tol=0.1, omega = rich_param, max_iter = 1)


    return im

def im_product(im1, im2):
    return np.tensordot(im1,im2)

def cg(op, b, im0, tol, max_iter = 1000):
    im = im0
    res = op(im0)-b
    p = -res
    for _ in range(max_iter):
        po = op(p)
        alpha = -im_product(res,p)/im_product(po,p)
        im = im + alpha*p
        res = op(im)-b
        beta = im_product(res,po)/im_product(po,p)
        p = -res + beta*p
        print(norm(res))
        if norm(res)<tol:
            break
    alpha = -im_product(res,p)/im_product(op(p),p)
    im = im + alpha*p
    return im



In [52]:
testim = cv2.imread("../images/originals/Baseline.jpg", 0).astype(float)
testim = cv2.resize(testim, (0,0), fx = 1, fy = 1)
regularized_im = cg(op, testim , testim, 0.01)
im_r = regularized_im.astype(np.uint8)
cv2.imshow("v",im_r)
cv2.waitKey(0)
cv2.destroyAllWindows()

826.8231936624679
123.48778492163741
17.640694225156235
2.510667982073799
0.3629437137038764
0.05229716285036945
0.007564231139858658


In [71]:
testim = cv2.imread("../images/originals/Baseline.jpg", 0).astype(float)
testim = cv2.resize(testim, (0,0), fx = 0.1, fy = 0.1)
regularized_im = split_bregman(testim,cg, lhsoperator)
im_r = regularized_im.astype(np.uint8)
cv2.imshow("v",im_r)
cv2.waitKey(0)
cv2.destroyAllWindows()

6177.388939190596
2011.7651120718085
889.5446449977042
397.84897151434217
198.9916241137067
98.30139665687598
49.80579928480822
24.60870383353136
12.604366549712642
6.256445524117629
3.133759628212027
1.5547199920125931
0.790532725249513
0.39102378363863577
0.19463291835809085
0.0960520372085147
1543.61959996334
717.4139304390254
348.4246467747953
176.52593679320037
90.13505920071161
45.169066769819445
22.747965523779666
11.562888232180322
5.78665476511911
2.8807643901931352
1.4482936288578472
0.7318345634027585
0.3642894787041577
0.1801449443331299
0.09027744970711529
1031.6953417813575
390.7220570841774
170.66654474117783
76.15855466109599
37.73652225809117
18.78931282794998
9.77251860325283
5.096021807724846
2.709140366377238
1.3844155544396282
0.7059889969533939
0.35713443453068067
0.18167650531205196
0.08972788575304377
757.4798890807609
251.870072668208
103.9541474280864
44.14547956569756
22.10972372559711
11.731584580216584
6.574407451564182
3.5026909813215794
1.8433994514014522

In [7]:
im1 = da.Image("../images/originals/Baseline.jpg", (0,0), 10, 100)

im2 = im1.copy()

In [3]:
im1.origo

(0, 0)

In [8]:
im2.origo

(0, 0)

In [9]:
im1.origo = (1,0)

In [10]:
im2.origo

(0, 0)